In [1]:
from dotenv import load_dotenv
import os

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content="I'm unable to provide real-time weather updates or forecasts. To get the current weather in Munich, I recommend checking a reliable weather website or a weather app for the most accurate and up-to-date information.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 17, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6i37PzEgaqheqKqynO7oeY2pl52p', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c2cf6d3f-25ea-40ce-8084-23a060e489b1-0', usage_metadata={'input_tokens': 17, 'output_tokens': 40, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

In [4]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class WeatherCheck(BaseModel):
#     """Check the weather in a specified city."""

#     city: str = Field(..., description="Name of the city to check the weather for")


# class OutdoorSeatingCheck(BaseModel):
#     """Check if outdoor seating is available at a specified restaurant in a given city."""

#     city: str = Field(..., description="Name of the city where the restaurant is located")


# tools = [WeatherCheck, OutdoorSeatingCheck]

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lc3RT3YDtil6Ec10vHhZxUf0', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 158, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6i7MJ8EPrbX6XtBIMy2c2A6qqD4V', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--37a2c606-2837-4e27-88d6-c06949966264-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_lc3RT3YDtil6Ec10vHhZxUf0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 158, 'output_tokens': 16, 'total_tokens': 174, 'input_token_de

In [7]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xJdR5OB9sfvIfxAOCF67SXLK', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_0TWeufgBhk9iZ2yDHtxiBCb9', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6i8RAEVxg6SoxwEJNntjnbKCmhLY', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c2bfac91-ae51-4a70-a866-31a055b2341e-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id':

In [8]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Munich'},
  'id': 'call_xJdR5OB9sfvIfxAOCF67SXLK',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Munich'},
  'id': 'call_0TWeufgBhk9iZ2yDHtxiBCb9',
  'type': 'tool_call'}]

In [9]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [10]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oHYW9F4D8SAhOMik5cxK0cYs', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_08GMIZzgr8C9xJQg1qdjl3yX', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6i9Vi2SbcjO4MxA2MTKv385idk0f', 'service_tier': 'default', 'finish_reason': 'tool

In [11]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [12]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [13]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oHYW9F4D8SAhOMik5cxK0cYs', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_08GMIZzgr8C9xJQg1qdjl3yX', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6i9Vi2SbcjO4MxA2MTKv385idk0f', 'service_tier': 'default', 'finish_reason': 'tool

In [14]:
llm_with_tools.invoke(messages)

AIMessage(content="Today's weather in Munich is sunny with a temperature of 22°C. Additionally, outdoor seating is available, so you can enjoy your meal outside!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 245, 'total_tokens': 275, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6iBQ6fOtv7mAWXWQSJJtUEBrq8My', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--da6ef409-487d-47cf-92f9-f12e3516f6ad-0', usage_metadata={'input_tokens': 245, 'output_tokens': 30, 'total_tokens': 275, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Real third party API Call

In [15]:
from langchain_core.tools import tool
import httpx



@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city from a FastAPI endpoint on localhost:8000."""
    response = httpx.get(f"http://localhost:5566/weather/{city}")

    if response.status_code == 200:
        return response.json().get("weather", "Weather information not available")
    else:
        return "Failed to get weather information"


@tool
def outdoor_seating_availability(city: str) -> str:
    """Check if outdoor seating is available in a specified city from a FastAPI endpoint on localhost:8000."""
    response = httpx.get(f"http://localhost:5566/outdoor-seating/{city}")

    if response.status_code == 200:
        return response.json().get(
            "outdoor_seating", "Outdoor seating information not available"
        )
    else:
        return "Failed to get outdoor seating information"


api_tools = [fake_weather_api, outdoor_seating_availability]
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [16]:
from langchain_openai import ChatOpenAI


def interact_with_llm_and_tools(human_message: str):

    llm = ChatOpenAI(model="gpt-4o-mini")
    llm_with_tools_new = llm.bind_tools(api_tools)

    messages = [HumanMessage(content=human_message)]

    llm_output = llm_with_tools_new.invoke(messages)
    messages.append(llm_output)

    print("TOOL_CALLS:", llm_output.tool_calls)

    for tool_call in llm_output.tool_calls:
        tool_name = tool_call["name"].lower()
        tool = tool_mapping.get(tool_name)


        if tool:
            tool_output = tool.invoke(tool_call["args"])
            print("TOOL OUTPUT:", tool_output)
            messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    final_response = llm_with_tools_new.invoke(messages)
    return final_response

In [17]:
interact_with_llm_and_tools(
    "How will the weather be in sunnyville today? I would like to eat outside if possible"
)

TOOL_CALLS: [{'name': 'fake_weather_api', 'args': {'city': 'sunnyville'}, 'id': 'call_xXd5L9j2gK4D1aN0j97QkSDw', 'type': 'tool_call'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'sunnyville'}, 'id': 'call_yG9N07NO3msojjnQv3uRXUFD', 'type': 'tool_call'}]
TOOL OUTPUT: Sunny, 25°C
TOOL OUTPUT: Outdoor seating is available.


AIMessage(content='The weather in Sunnyville today is sunny with a temperature of 25°C. Additionally, outdoor seating is available, so you can enjoy your meal outside!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 193, 'total_tokens': 225, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6iGf2xdrKGcQLPwXn7u96ayWYzyM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--cc469564-671d-4408-aee7-111dde01ac53-0', usage_metadata={'input_tokens': 193, 'output_tokens': 32, 'total_tokens': 225, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
interact_with_llm_and_tools(
    "How will the weather be in rainytown today? I would like to eat outside if possible"
)

TOOL_CALLS: [{'name': 'fake_weather_api', 'args': {'city': 'rainytown'}, 'id': 'call_4sM9JYggMUCacjtX3R8DKDpG', 'type': 'tool_call'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'rainytown'}, 'id': 'call_haXzdBrQxukdiBqjr7LJvJVu', 'type': 'tool_call'}]
TOOL OUTPUT: Rainy, 16°C
TOOL OUTPUT: Outdoor seating is not available.


AIMessage(content='The weather in Rainytown today is rainy with a temperature of 16°C. Unfortunately, outdoor seating is not available.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 196, 'total_tokens': 222, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6iGwZTHnTDegjUWRknMWqdXqaKLA', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--50737992-dab3-4d90-b1fb-ca54bb0e2232-0', usage_metadata={'input_tokens': 196, 'output_tokens': 26, 'total_tokens': 222, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
interact_with_llm_and_tools(
    "How will the weather be in munich today? I would like to eat outside if possible"
)

TOOL_CALLS: [{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': 'call_5Uk75q2RBRHbi9oBC6ikbjeM', 'type': 'tool_call'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'munich'}, 'id': 'call_PaeC82zxQChPulfPIUbPsdnb', 'type': 'tool_call'}]
TOOL OUTPUT: Sunny, 22°C
TOOL OUTPUT: Outdoor seating is available.


AIMessage(content='The weather in Munich today is sunny with a temperature of 22°C. Additionally, outdoor seating is available, so it would be nice to eat outside!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 191, 'total_tokens': 223, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6iH7iF27ow4j03VKKTDa8XArbS7I', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8b709b71-8239-428d-9fa3-9a7410bbf304-0', usage_metadata={'input_tokens': 191, 'output_tokens': 32, 'total_tokens': 223, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [20]:
interact_with_llm_and_tools("What´s the name of the Dad of the simpsons family?")

TOOL_CALLS: []


AIMessage(content='The name of the dad in the Simpson family is Homer Simpson.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 124, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C6iHFeZP6zL2cwC8UCyiOg0MsR7YR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9ec55fca-66fa-47ae-8611-a595a7687ae3-0', usage_metadata={'input_tokens': 124, 'output_tokens': 14, 'total_tokens': 138, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})